In [5]:
import pandas as pd
import numpy as np
import yfinance as yf
from backtest import Backtest, DisplayBacktest, Strategy

In [6]:
class MovingAverageStrategy(Strategy):
    def __init__(self, name, data, short_window, long_window):
        super().__init__(name, data)
        self.short_window = short_window
        self.long_window = long_window
        self.tickers = data.columns

        # Compute moving averages for each ticker
        for ticker in self.tickers:
            self.data[f'{self.strategy_name}_{ticker}_short_ma'] = self.data[ticker].rolling(window=self.short_window).mean()
            self.data[f'{self.strategy_name}_{ticker}_long_ma'] = self.data[ticker].rolling(window=self.long_window).mean()

    def calculate_signal(self):
        for ticker in self.tickers:
            self.data[f'{self.strategy_name}_{ticker}_signal'] = 0.0
            self.data[f'{self.strategy_name}_{ticker}_signal'][self.short_window:] = np.where(
                self.data[f'{self.strategy_name}_{ticker}_short_ma'][self.short_window:] > self.data[f'{self.strategy_name}_{ticker}_long_ma'][self.short_window:], 1.0, 0.0
            )

    def calculate_positions(self):
        positions_list = []
        for ticker in self.tickers:
            positions = pd.DataFrame({
                'time': self.data.index,
                'book': self.strategy_name,
                'ticker': ticker,
                'units': np.where(self.data[f'{self.strategy_name}_{ticker}_signal'] == 1.0, 10, -10)
            })
            positions_list.append(positions)
        return pd.concat(positions_list)


In [9]:
# Define tickers and date range
tickers = ['AAPL', 'OXY']
start_date = '2015-01-01'
end_date = '2024-01-01'

# Fetch data
data = yf.download(tickers, start=start_date, end=end_date)['Close']

# Initialize and run backtest
strategy1 = MovingAverageStrategy(name="Strategy1", data=data, short_window=40, long_window=100)
strategy2 = MovingAverageStrategy(name="Strategy2", data=data, short_window=20, long_window=50)
strategies = [strategy1, strategy2]
weights = {"Strategy1": 0.7, "Strategy2": 0.3}

backtest = Backtest(data, strategies, weights)

# Plot results
display = DisplayBacktest(backtest)
display.plot_book("Strategy1")
display.plot_cumulative_pnl_per_book()
display.plot_cumulative_pnl()
display.plot_individual_pnl()

# Export to Excel
backtest.export_excel("backtest_results.xlsx")

[*********************100%%**********************]  2 of 2 completed


Backtest results exported to backtest_results.xlsx
